In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib #for fuller functionality, incl. colors
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GMM
from matplotlib.colors import LogNorm
import pandas as pd
from IPython.display import display
import csv
import requests
from BeautifulSoup import BeautifulSoup
from bs4 import BeautifulSoup, Comment
import urllib2
import html2text
import os.path

# Remove Deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# for nicer printing
from prettytable import PrettyTable

#increase number of columns can see in pandas
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

**LOAD DF, GET RID OF "CANDIDATES" WITH ZERO VOTES**

In [2]:
df = pd.DataFrame.from_csv("text_files/senate_results_EDA_states_all_years.csv")

#clean up zero-value "candidates"

df = df[df['votes_state']!=0]

display(df.head(5))
display(df[(df['year']==2014) & (df['state_abbr']=="OK")].head())

,year,candidate,state_abbr,votes,votes_state,max_state,%_votes_state,winner
0,2010,Alexander Giannoulias,IL,1719478.0,3703901,1778698,46.4%,0
1,2010,Alvin Greene,SC,364598.0,1318794,810771,27.6%,0
3,2010,Barbara Boxer,CA,5218441.0,10000093,5218441,52.2%,1
4,2010,Barbara Mikulski,MD,1140531.0,1833304,1140531,62.2%,1
7,2010,Billy Wilson,KY,1214.0,1356468,755411,0.1%,0


,year,candidate,state_abbr,votes,votes_state,max_state,%_votes_state,winner
290,2014,Aaron DeLozier,OK,7786.0,819679,557537,0.9%,0
375,2014,James Inhofe,OK,557537.0,819679,557537,68.0%,1
386,2014,Joan Farr,OK,10534.0,819679,557537,1.3%,0
421,2014,Matt Silverstein,OK,233932.0,819679,557537,28.5%,0
446,2014,Ray Woods,OK,9890.0,819679,557537,1.2%,0


Idea: apply URL as best as possible from name
for those candidates with > 5% vote share that didn't match to a url, see if can get a match
For all that match, try to pull table

**Determine which names need manual correction**

In [3]:
unmatched_names_dict = {
"Mike Crapo":"Michael_Crapo",
"Bernard Sanders":"Bernie_Sanders",
"John Reed":"Jack_Reed",
"Thomas Carper":"Tom_Carper",
"Mike Crapo":"Michael_Crapo",
"Charles Grassley":"Chuck_Grassley",
"Shelley Capito":"Shelley_Moore_Capito",
"Edward Markey":"Ed_Markey",
"Joe Manchin":"Joe_Manchin_III",
"Jonathan Dine":"Jonathan_Dine",
"Christopher Coons":"Chris_Coons",
"Dan Coats":"Daniel_Coats",
"Robert Casey":"Bob_Casey",
"Timothy Kaine":"Tim_Kaine",
"Russ Feingold":"Russell_Feingold",
"Denny Rehberg":"Dennis_Rehberg",
"William Cassidy":"Bill_Cassidy",
"Alison Lundergan Grimes":"Alison_Grimes",
"Charlie Melancon":"Charles_Melancon",
"Michael Thurmond":"Mike_Thurmond",
"Pete Hoekstra":"Peter_Hoekstra",
"Blanche Lincoln":"Blanche_Lambert_Lincoln",
"Charles Summers":"Charlie_Summers",
"Edward Clifford":"edward-clifford-iii",
"Alexander Pires":"Alex_Pires",
"Alexander Giannoulias":"Alexi_Giannoulias",
"Daniel Bongino":"Dan_Bongino",
"John Kennedy":"John_Neely_Kennedy"
}

** CREATE FULL NAME BY REPLACING SPACES TO URL AND MANUALLY MATCHING NAMES (IF APPROPRIATE DUE TO DIFFERENT URL NAMING CONVENTION **

In [4]:
# add full name
def create_full_name(name) :
    if name in unmatched_names_dict.keys() :
        return unmatched_names_dict[name]
    else :
        return name.replace(" ","_")

df['full_name'] = df['candidate'].apply(create_full_name)
df['full_name_year'] = df['full_name']+"_"+df['year'].map(str)

display(df.head())
display(df)
# turn % votes to a decimal
# df['%_votes_state'] = df['%_votes_state'].apply(string_to_dec)


,year,candidate,state_abbr,votes,votes_state,max_state,%_votes_state,winner,full_name,full_name_year
0,2010,Alexander Giannoulias,IL,1719478.0,3703901,1778698,46.4%,0,Alexi_Giannoulias,Alexi_Giannoulias_2010
1,2010,Alvin Greene,SC,364598.0,1318794,810771,27.6%,0,Alvin_Greene,Alvin_Greene_2010
3,2010,Barbara Boxer,CA,5218441.0,10000093,5218441,52.2%,1,Barbara_Boxer,Barbara_Boxer_2010
4,2010,Barbara Mikulski,MD,1140531.0,1833304,1140531,62.2%,1,Barbara_Mikulski,Barbara_Mikulski_2010
7,2010,Billy Wilson,KY,1214.0,1356468,755411,0.1%,0,Billy_Wilson,Billy_Wilson_2010


,year,candidate,state_abbr,votes,votes_state,max_state,%_votes_state,winner,full_name,full_name_year
0,2010,Alexander Giannoulias,IL,1719478.0,3703901,1778698,46.4%,0,Alexi_Giannoulias,Alexi_Giannoulias_2010
1,2010,Alvin Greene,SC,364598.0,1318794,810771,27.6%,0,Alvin_Greene,Alvin_Greene_2010
3,2010,Barbara Boxer,CA,5218441.0,10000093,5218441,52.2%,1,Barbara_Boxer,Barbara_Boxer_2010
4,2010,Barbara Mikulski,MD,1140531.0,1833304,1140531,62.2%,1,Barbara_Mikulski,Barbara_Mikulski_2010
7,2010,Billy Wilson,KY,1214.0,1356468,755411,0.1%,0,Billy_Wilson,Billy_Wilson_2010
8,2010,Blanche Lincoln,AR,288156.0,779957,451618,36.9%,0,Blanche_Lambert_Lincoln,Blanche_Lambert_Lincoln_2010
10,2010,Brad Ellsworth,IN,697775.0,1744221,952116,40.0%,0,Brad_Ellsworth,Brad_Ellsworth_2010
16,2010,Cam Cavasso,HI,79924.0,370506,277167,21.6%,0,Cam_Cavasso,Cam_Cavasso_2010
18,2010,Carly Fiorina,CA,4217366.0,10000093,5218441,42.2%,0,Carly_Fiorina,Carly_Fiorina_2010
19,2010,Charles Grassley,IA,718215.0,1116063,718215,64.4%,1,Chuck_Grassley,Chuck_Grassley_2010


**GET INFORMATION AND WRITE TO TEXT FILES FOR SUBSQUENT ANALYSIS  
ALSO, CHECK UNMATCHED NAMES**

In [5]:
### GET URLs, WRITE TO FILE ###
#~~UNCOMMENT TO ACTUALLY WRITE~~#

url_names = {}
unmatched_names = []

for full_name, candidate, results, state in zip(df['full_name'],df['candidate'], df['%_votes_state'], df['state_abbr']) :
    try :
        #if file already exists, skip this to take less time
        if os.path.isfile('text_files/'+full_name+'.txt') :
            continue
        
        url = 'http://www.ontheissues.org/Senate/'+full_name+'.htm'
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html)
        page = urllib2.urlopen(url)
        html_content = page.read()
        #ignore errors to get to write
        html_content = unicode(html_content, errors='ignore')
        rendered_content = html2text.html2text(html_content)
        
        #write to text file to make scrapable
        with open('text_files/'+full_name+'.txt', 'w') as f :
            f.write(rendered_content)
        f.close()
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        print "GOOD:", candidate, state, url, results
    except :
        unmatched_names.append((candidate, results, state))
        print "BAD:", candidate, state, results

C:\Users\kowen\AppData\Local\Continuum\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


GOOD: Alexander Giannoulias IL http://www.ontheissues.org/Senate/Alexi_Giannoulias.htm 46.4%
GOOD: Alvin Greene SC http://www.ontheissues.org/Senate/Alvin_Greene.htm 27.6%
GOOD: Barbara Boxer CA http://www.ontheissues.org/Senate/Barbara_Boxer.htm 52.2%
GOOD: Barbara Mikulski MD http://www.ontheissues.org/Senate/Barbara_Mikulski.htm 62.2%
BAD: Billy Wilson KY 0.1%
GOOD: Blanche Lincoln AR http://www.ontheissues.org/Senate/Blanche_Lambert_Lincoln.htm 36.9%
GOOD: Brad Ellsworth IN http://www.ontheissues.org/Senate/Brad_Ellsworth.htm 40.0%
GOOD: Cam Cavasso HI http://www.ontheissues.org/Senate/Cam_Cavasso.htm 21.6%
GOOD: Carly Fiorina CA http://www.ontheissues.org/Senate/Carly_Fiorina.htm 42.2%
GOOD: Charles Grassley IA http://www.ontheissues.org/Senate/Chuck_Grassley.htm 64.4%
GOOD: Charles Schumer NY http://www.ontheissues.org/Senate/Charles_Schumer.htm 66.3%
BAD: Charley Miller CO 5.2%
GOOD: Charlie Crist FL http://www.ontheissues.org/Senate/Charlie_Crist.htm 30.5%
GOOD: Charlie Melanco

In [6]:
#SHOW REMAINING UNMATCHED NAMES
unmatched_names = sorted(unmatched_names, key=lambda x: (x[1]), reverse=True)
for name, result, state in unmatched_names :
    print '"'+name+'":,', result, state
    
unmatched_names_name_only = [i[0] for i in unmatched_names]

"Curt Gottshall":, 8.0% WY
"Paul Strauss":, 77.3% DC
"Glenda Richmond":, 7.0% DC
"David Nolan":, 6.7% AZ
"Dan Cox":, 6.5% MT
"Ray Writz":, 6.2% ID
"Jonathan Dine":, 6.1% MO
"Andrew Horning":, 5.8% IN
"Scott Bradley":, 5.7% UT
"Robert Zadek":, 5.6% IL
"Gary Swing":, 5.5% AZ
"Lucy Brenton":, 5.5% IN
"Robert Garrard":, 5.5% KS
"Rebecca Sink-Burris":, 5.4% IN
"Charley Miller":, 5.2% CO
"Trevor Drown":, 5.2% AR
"Jonathan Dine":, 5.1% MO
"John Daniel":, 5.0% DC
"Thomas Smith":, 44.7% PA
"Albert Gore":, 40.3% MS
"David VanDerBeek":, 4.9% NV
"Scott Rupert":, 4.6% OH
"None of these candidates":, 4.5% NV
"Marc Victor":, 4.4% AZ
"Randy Bergquist":, 3.9% ID
"Edward Clifford":, 3.9% PA
"None of these candidates":, 3.8% NV
"Mark Fish":, 3.7% AK
"Jeff Jarrett":, 3.6% HI
"Jon Barrie":, 3.6% NM
"Michael Dann":, 3.5% KS
"Lily Williams":, 3.5% CO
"Chris Booth":, 3.2% NH
"Kenton McMillen":, 3.2% IL
"Shaun McCausland":, 3.1% UT
"David Patterson":, 3.1% KY
"Shanti Lewallen":, 3.1% OR
"Bob Baber":, 3.0% WV
"

**REMOVE "BAD" NAMES FROM DF, ADD IN URL_NAME**

In [7]:
#add the url to the df
def add_url_name(underscore_name) :
    return 'http://www.ontheissues.org/Senate/'+underscore_name+'.htm'

df['url_name'] = df['full_name'].apply(add_url_name)

#determine whether is a matched or unmatched name
def matched_names(candidate_name) :
    if candidate_name in unmatched_names_name_only :
        return "N"
    else :
        return "Y"

print matched_names("Albert Gore")
    
#add to df
df['matched_name'] = df['candidate'].apply(matched_names)

#get rid of unmatched names from data frame as will not be able to pull in any relevant information
df = df[df['matched_name']=="Y"]
    
display(df.head())
display(df)

N


,year,candidate,state_abbr,votes,votes_state,max_state,%_votes_state,winner,full_name,full_name_year,url_name,matched_name
0,2010,Alexander Giannoulias,IL,1719478.0,3703901,1778698,46.4%,0,Alexi_Giannoulias,Alexi_Giannoulias_2010,http://www.ontheissues.org/Senate/Alexi_Gianno...,Y
1,2010,Alvin Greene,SC,364598.0,1318794,810771,27.6%,0,Alvin_Greene,Alvin_Greene_2010,http://www.ontheissues.org/Senate/Alvin_Greene...,Y
3,2010,Barbara Boxer,CA,5218441.0,10000093,5218441,52.2%,1,Barbara_Boxer,Barbara_Boxer_2010,http://www.ontheissues.org/Senate/Barbara_Boxe...,Y
4,2010,Barbara Mikulski,MD,1140531.0,1833304,1140531,62.2%,1,Barbara_Mikulski,Barbara_Mikulski_2010,http://www.ontheissues.org/Senate/Barbara_Miku...,Y
8,2010,Blanche Lincoln,AR,288156.0,779957,451618,36.9%,0,Blanche_Lambert_Lincoln,Blanche_Lambert_Lincoln_2010,http://www.ontheissues.org/Senate/Blanche_Lamb...,Y


,year,candidate,state_abbr,votes,votes_state,max_state,%_votes_state,winner,full_name,full_name_year,url_name,matched_name
0,2010,Alexander Giannoulias,IL,1719478.0,3703901,1778698,46.4%,0,Alexi_Giannoulias,Alexi_Giannoulias_2010,http://www.ontheissues.org/Senate/Alexi_Gianno...,Y
1,2010,Alvin Greene,SC,364598.0,1318794,810771,27.6%,0,Alvin_Greene,Alvin_Greene_2010,http://www.ontheissues.org/Senate/Alvin_Greene...,Y
3,2010,Barbara Boxer,CA,5218441.0,10000093,5218441,52.2%,1,Barbara_Boxer,Barbara_Boxer_2010,http://www.ontheissues.org/Senate/Barbara_Boxe...,Y
4,2010,Barbara Mikulski,MD,1140531.0,1833304,1140531,62.2%,1,Barbara_Mikulski,Barbara_Mikulski_2010,http://www.ontheissues.org/Senate/Barbara_Miku...,Y
8,2010,Blanche Lincoln,AR,288156.0,779957,451618,36.9%,0,Blanche_Lambert_Lincoln,Blanche_Lambert_Lincoln_2010,http://www.ontheissues.org/Senate/Blanche_Lamb...,Y
10,2010,Brad Ellsworth,IN,697775.0,1744221,952116,40.0%,0,Brad_Ellsworth,Brad_Ellsworth_2010,http://www.ontheissues.org/Senate/Brad_Ellswor...,Y
16,2010,Cam Cavasso,HI,79924.0,370506,277167,21.6%,0,Cam_Cavasso,Cam_Cavasso_2010,http://www.ontheissues.org/Senate/Cam_Cavasso.htm,Y
18,2010,Carly Fiorina,CA,4217366.0,10000093,5218441,42.2%,0,Carly_Fiorina,Carly_Fiorina_2010,http://www.ontheissues.org/Senate/Carly_Fiorin...,Y
19,2010,Charles Grassley,IA,718215.0,1116063,718215,64.4%,1,Chuck_Grassley,Chuck_Grassley_2010,http://www.ontheissues.org/Senate/Chuck_Grassl...,Y
21,2010,Charles Schumer,NY,3047775.0,4596589,3047775,66.3%,1,Charles_Schumer,Charles_Schumer_2010,http://www.ontheissues.org/Senate/Charles_Schu...,Y


**GET NUMBER OF RESPONSES PER TOPIC DESCRIPTION  
DO THIS TO SEE WHETHER TOPIC TYPES ARE STILL RELEVANT**

In [8]:
### GET URL, WRITE TO FILE ###
#~~UNCOMMENT TO ACTUALLY WRITE~~#


all_topics_list = []

all_data_list = []

#for each relevant candidate, get to relevant information
for name, year in zip(df['full_name'], df['year']) :
    
    with open('text_files/'+name+'.txt') as f :
        text = f.readlines()
    f.close()

    topic_start = False
    data_list = []

    poss_topics = ["topic 1:", "topic 2:","topic 3:","topic 4:","topic 5:","topic 6:","topic 7:","topic 8:",
                  "topic 9:","topic 10:","topic 11:","topic 12:","topic 13:","topic 14:","topic 15:","topic 16:",
                  "topic 17:","topic 18:","topic 19:","topic 20:"]

    topics_dict = {}
    keep_lines = False
    topic_type = None
    priorline_topic = None

    for line in text :
        
        if "topic 1:" in line :
            topic_start = True

        #if are in topics section...
        if topic_start == True :
            
#             print line 
#             if topic == "1" :
#                 print name, topic_type, topic_descrip, score, line 
            
            #get 2nd line of topic description
            if keep_lines == True :
                #append until hit closing "]" in the 2nd line
                topic_descrip = topic_descrip+" "+line[:line.find("]")]
                topics_dict[topic] = topic_descrip
                keep_lines = False


            #get points and topic type for the question 
            if "points on social scale" in line.lower() or "points on economic scale" in line.lower() :
                score = int(line[line.find("(")+1:line.find("points")-1])

                #get social vs economic type
                if "points on social scale" in line.lower() :
                    topic_type = "Social"
                    ####### IMPLEMENT AGAIN AFTER ONCE DEAL WITH REGULAR TOPIC TYPES!!!!!!!!! #######
                    #if social, multiple by -1 to also make more left-wing more negative like economic scale
#                     score = -1*score
                else :
                    topic_type = "Economic"
            
                #append all the info from the prior topic to all_data_list
                all_data_list.append((name, year, topic, topic_descrip, topic_type, score))


            #if topic number in line...
            if any(t in line for t in poss_topics):
                
                #get the topic number
                topic = line[line.find("topic")+6:line.find(":")]

                #if "]" not in line than topic description is spaced over two lines...
                if "]" not in line :
                    #get the start of topic_descrip to combine later
                    topic_descrip = line[line.find(":")+2:].strip()
                    #keep a line for drawdown
                    keep_lines = True
                else:
                    #pull topic_description fully from current line
                    topic_descrip = line[line.find(":")+2:line.find("]")].strip()
                    topics_dict[topic] = topic_descrip
                    keep_lines = False

            #reset: don't read two lines at once for topic decr any longer as long as not starting a new topic    
            else:
                keep_lines = False

#get all the info from final topic otherwise missed on final loop
#append all the info from the FINAL topic to all_data_list
all_data_list.append((name, year, topic, topic_descrip, topic_type, score))

#get rid of dupes
all_data_list = list(set(all_data_list))

**Display all topics in DataFrame to make simlarity determinations**

In [9]:
############ GET RID OF ALL TOPICS LIST AFTER ############

# print len(all_topics_list)
# print sorted(all_topics_list)

df_topics = pd.DataFrame(all_data_list)
df_topics.columns = ['full_name', 'year', 'topic', 'topic_descrip','topic_type','score']

#concatenate a column for name and year
df_topics['full_name_year'] = df_topics['full_name']+"_"+df_topics['year'].map(str)

def return_max(full_name_year) :
    return df_topics[df_topics['full_name_year']==full_name_year]['score'].max()

### RE-IMPLEMENT FOR MAX_SCORE ###
df_topics['max_score'] = df_topics['full_name_year'].apply(return_max)

### ASSIGN NORMALIZED SCORE ###
def normalize_score(row) :
    #if person is on a scale of 10 instead of 5, then reduce everything by 5
    if row['max_score'] == 10:
        return row['score'] - 5
    else :
        return row['score']
    
df_topics['normalized_score'] = df_topics.apply(normalize_score, axis=1)


### FLIP SIGN OF SOCIAL SCORE TO MAKE LIKE ECONOMIC; LEFT-WING == NEGATIVE, RIGHT-WING = POSITIVE ###
def flip_social(row) :
    #if person is on a scale of 10 instead of 5, then reduce everything by 5
    if row['topic_type'] == "Social":
        return -1*row['normalized_score']
    else :
        return row['normalized_score']

df_topics['updated_score'] = df_topics.apply(flip_social, axis=1)

display(df_topics)



,full_name,year,topic,topic_descrip,topic_type,score,full_name_year,max_score,normalized_score,updated_score
0,Wendy_Long,2012,19,Marijuana is a gateway drug,Social,-5,Wendy_Long_2012,5,-5,5
1,Alexi_Giannoulias,2010,20,Stimulus better than market-led recovery,Economic,5,Alexi_Giannoulias_2010,5,5,5
2,Joe_Kyrillos,2012,2,Require hiring more women & minorities,Economic,2,Joe_Kyrillos_2012,5,2,2
3,Cory_Booker,2014,18,Prioritize green energy,Economic,-5,Cory_Booker_2014,5,-5,-5
4,Thad_Cochran,2014,20,Stimulus better than market-led recovery,Economic,2,Thad_Cochran_2014,5,2,2
5,Benjamin_Cardin,2012,9,Stricter punishment reduces crime,Social,2,Benjamin_Cardin_2012,5,2,-2
6,Jim_Huffman,2010,13,Support & expand free trade,Economic,5,Jim_Huffman_2010,5,5,5
7,Ron_Johnson,2016,9,Stricter punishment reduces crime,Social,2,Ron_Johnson_2016,5,2,-2
8,Maggie_Hassan,2016,13,Support & expand free trade,Economic,2,Maggie_Hassan_2016,5,2,2
9,Mitch_McConnell,2014,19,Marijuana is a gateway drug,Social,-5,Mitch_McConnell_2014,5,-5,5


In [10]:
df_topics_groupby = df_topics.groupby(['topic', 'topic_descrip']).size().reset_index(name='counts')
display(df_topics_groupby)

# df_topics_describe = df_topics.groupby(['topic', 'topic_descrip']).describe()
# display(df_topics_describe)

,topic,topic_descrip,counts
0,1,Abortion is a woman's right,59
1,1,Abortion is a woman's unrestricted right,245
2,10,Absolute right to gun ownership,304
3,11,Decrease overall taxation of the wealthy,2
4,11,Higher taxes on the wealthy,245
5,11,Make taxes more progressive,57
6,12,Illegal immigrants earn citizenship,57
7,12,Immigration helps our economy-encourage it,2
8,12,Pathway to citizenship for illegal aliens,245
9,13,Support & expand free trade,304


In [11]:
# display(df_topics[df_topics['max_score']==10])

In [12]:
aligned_descriptions = [
"Abortion is a woman's right",
"Abortion is a woman's unrestricted right",
"Absolute right to gun ownership",
"Decrease overall taxation of the wealthy",
"Higher taxes on the wealthy",
"Make taxes more progressive",
"Illegal immigrants earn citizenship",
"Immigration helps our economy-encourage it",
"Pathway to citizenship for illegal aliens",
"Support & expand free trade",
"Maintain US sovereignty from UN",
"Support American Exceptionalism",
"Expand the armed forces",
"Expand the military",
"More spending on armed forces",
"Make voter registration easier",
"Avoid foreign entanglements",
"Seek UN approval for military action",
"Stay out of Iran",
"US out of Iraq",
"US out of Iraq & Afghanistan",
"Prioritize green energy",
"Reduce use of coal, oil, & nuclear energy",
"Replace coal & oil with alternatives",
"Drug use is immoral: enforce laws against it",
"Marijuana is a gateway drug",
"Never legalize marijuana",
"Legally require hiring women & minorities",
"Require companies to hire more women & minorities",
"Require hiring more women & minorities",

"Stimulus better than market-led recovery",
"Comfortable with same-sex marriage",
"Same-sex domestic partnership benefits",
"Sexual orientation protected by civil rights laws",
"Keep God in the public sphere",
"Permit prayer in public schools",
"Teacher-led prayer in public schools",
"Expand ObamaCare",
"More federal funding for health coverage",
"Privatize Social Security",
"Parents choose schools via vouchers",
"Vouchers for school choice",
"EPA regulations are too restrictive",
"Society bears cost of pollution",
"Mandatory Three Strikes sentencing laws",
"Stricter punishment reduces crime",
]


#totally different topics; not relevant to main topic
unaligned_descriptions = [
"The Patriot Act harms civil liberties",
"Link human rights to trade with China",
"Reduce spending on missile defense (Star Wars)",
"Stricter limits on political campaign funds",
"Death Penalty",
"Human needs over animal rights",
"Allow churches to provide welfare services",
]



**REPLACE ANSWERS WITH NaN FOR ANY UNALIGNED DESCRIPTIONS**

In [13]:
def topic_alignment(topic_descrip) :
    if topic_descrip in unaligned_descriptions :
        return "N"
    else :
        return "Y"
    
df_topics['aligned_topic'] = df_topics['topic_descrip'].apply(topic_alignment)


### ASSIGN NaN TO UPDATED SCORE IF THE TOPIC IS NOT ALIGNED ###
def replace_with_NaN(row) :
    if row['topic_descrip'] in unaligned_descriptions :
        return np.nan
    else :
        return row['updated_score']

df_topics['updated_score'] = df_topics.apply(replace_with_NaN, axis=1)

display(df_topics)

,full_name,year,topic,topic_descrip,topic_type,score,full_name_year,max_score,normalized_score,updated_score,aligned_topic
0,Wendy_Long,2012,19,Marijuana is a gateway drug,Social,-5,Wendy_Long_2012,5,-5,5.0,Y
1,Alexi_Giannoulias,2010,20,Stimulus better than market-led recovery,Economic,5,Alexi_Giannoulias_2010,5,5,5.0,Y
2,Joe_Kyrillos,2012,2,Require hiring more women & minorities,Economic,2,Joe_Kyrillos_2012,5,2,2.0,Y
3,Cory_Booker,2014,18,Prioritize green energy,Economic,-5,Cory_Booker_2014,5,-5,-5.0,Y
4,Thad_Cochran,2014,20,Stimulus better than market-led recovery,Economic,2,Thad_Cochran_2014,5,2,2.0,Y
5,Benjamin_Cardin,2012,9,Stricter punishment reduces crime,Social,2,Benjamin_Cardin_2012,5,2,-2.0,Y
6,Jim_Huffman,2010,13,Support & expand free trade,Economic,5,Jim_Huffman_2010,5,5,5.0,Y
7,Ron_Johnson,2016,9,Stricter punishment reduces crime,Social,2,Ron_Johnson_2016,5,2,-2.0,Y
8,Maggie_Hassan,2016,13,Support & expand free trade,Economic,2,Maggie_Hassan_2016,5,2,2.0,Y
9,Mitch_McConnell,2014,19,Marijuana is a gateway drug,Social,-5,Mitch_McConnell_2014,5,-5,5.0,Y


**CREATE NAMES PIVOT TABLE WITH ALL CRUCIAL INFORMATION**

In [14]:
#create pivot table where topic numbers are the columns to get candidate information in model-able order

df_names = pd.pivot_table(df_topics, values='updated_score', index=['full_name_year', 'full_name','year'],
                     columns=['topic'], aggfunc=np.sum)


#Reset index to be able to merge
df_names = df_names.reset_index()

#merge based on unique full_name_year
df_names = pd.merge(df_names,
                 df[['full_name_year','candidate', 'state_abbr', 'votes','votes_state',
                    'max_state','%_votes_state','winner','url_name']] ,
                 on='full_name_year',
              how='left',
                   )

#reorder column names
column_names = [
                'candidate', 'full_name', 'year', 'full_name_year', 'max_state', 'state_abbr', 
                'url_name', 'votes', 'votes_state', 'winner',
                '%_votes_state', '1', '2',  '3', '4', '5', '6', '7', '8', '9', '10',
    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']


#reset column order
df_names = df_names[column_names]

display(df_names.head())                   


topic,candidate,full_name,year,full_name_year,max_state,state_abbr,url_name,votes,votes_state,winner,%_votes_state,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Al Franken,Al_Franken,2014,Al_Franken_2014,1052794,MN,http://www.ontheissues.org/Senate/Al_Franken.htm,1052794.0,1980016,1,53.2%,-5.0,-3.0,-5.0,-2.0,-5.0,-5.0,-3.0,-2.0,-2.0,-5.0,-5.0,-5.0,-3.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
1,Alex Merced,Alex_Merced,2016,Alex_Merced_2016,4788374,NY,http://www.ontheissues.org/Senate/Alex_Merced.htm,43856.0,6799540,0,0.6%,-5.0,5.0,-5.0,-5.0,5.0,2.0,5.0,5.0,-5.0,5.0,5.0,-5.0,5.0,-3.0,-2.0,-5.0,-5.0,5.0,-5.0,5.0
2,Alexander Pires,Alex_Pires,2012,Alex_Pires_2012,265374,DE,http://www.ontheissues.org/Senate/Alex_Pires.htm,15300.0,399559,0,3.8%,-5.0,-5.0,-5.0,0.0,0.0,-5.0,-3.0,NaN,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
3,Alexander Giannoulias,Alexi_Giannoulias,2010,Alexi_Giannoulias_2010,1778698,IL,http://www.ontheissues.org/Senate/Alexi_Gianno...,1719478.0,3703901,0,46.4%,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-3.0,NaN,0.0,-5.0,-3.0,-5.0,-3.0,-5.0,3.0,NaN,-2.0,-5.0,0.0,5.0
4,Alison Lundergan Grimes,Alison_Grimes,2014,Alison_Grimes_2014,806795,KY,http://www.ontheissues.org/Senate/Alison_Grime...,584699.0,1435734,0,40.7%,-5.0,-5.0,-5.0,0.0,-5.0,-3.0,-3.0,3.0,3.0,2.0,-3.0,-2.0,-3.0,0.0,3.0,-2.0,-2.0,2.0,-2.0,-5.0


**MAKE CSV**

In [15]:
df_names.to_csv('text_files/candidate_info.csv')

**ISSUES RUN INTO:**
- Any document with Doctype declared "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "" target="_blank"http://www.w3.org/TR/html4/loose.dtd"> would not render text with html2text. Have to html_content = unicode(html_content, errors='ignore')  
- Different questions per year; have to see which ones are applicable
- Improper *ordering* of questions as well within a table, for example http://www.ontheissues.org/Senate/Blanche_Lambert_Lincoln.htm
- Different point scales (see http://www.ontheissues.org/Senate/Connie_Mack.htm) 